## 2장 파이썬 코드 연습

In [1]:
import pandas as pd

In [13]:
df = pd.DataFrame(data = [[1, 2017, 'gold'],[2, 2018, 'silver'],[3, 2016, 'inactive']], index = [0,1,2], columns = ['ID','Year','Status'])

In [14]:
df

,ID,Year,Status
0,1,2017,gold
1,2,2018,silver
2,3,2016,inactive


In [19]:
#기본적인 groupby 함수
df.groupby('ID').count()

,Year,Status
ID,,
1,1,1
2,1,1
3,1,1


In [20]:
df.groupby('ID').count().groupby('Status').count()

,Year
Status,
1,3


In [26]:
df = pd.DataFrame(data = [[1, 2017, 'gold'],[1,2018, 'gold'],[2, 2018, 'silver'],[3, 2016, 'inactive'],[3,2017,'silver'],[3,2018,'gold']], index = [0,1,2,3,4,5], columns = ['ID','Year','Status'])

In [27]:
df

,ID,Year,Status
0,1,2017,gold
1,1,2018,gold
2,2,2018,silver
3,3,2016,inactive
4,3,2017,silver
5,3,2018,gold


In [28]:
#기본적인 groupby 함수
df.groupby('ID').count()

,Year,Status
ID,,
1,2,2
2,1,1
3,3,3


In [29]:
df.groupby('ID').count().groupby('Status').count()

,Year
Status,
1,1
2,1
3,1


56 pg

In [74]:
import datetime

In [75]:
emails = pd.DataFrame(data = [[2,'2017-01-08',3,],[2,'2017-01-15',2],[1,'2017-01-15',1]],columns = ['member', 'week', 'EmailsOpened'])

In [76]:
emails.week = emails.week.apply(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d'))

In [77]:
emails

,member,week,EmailsOpened
0,2,2017-01-08,3
1,2,2017-01-15,2
2,1,2017-01-15,1


In [78]:
emails[emails.EmailsOpened < 1]

,member,week,EmailsOpened


In [79]:
emails[emails.member == 2]

,member,week,EmailsOpened
0,2,2017-01-08,3
1,2,2017-01-15,2


In [80]:
(max(emails[emails.member == 2].week)  -  min(emails[emails.member == 2].week)).days/7

1.0

In [81]:
complete_idx = pd.MultiIndex.from_product((set(emails.week),set(emails.member)))
complete_idx

MultiIndex([('2017-01-08', 1),
            ('2017-01-08', 2),
            ('2017-01-15', 1),
            ('2017-01-15', 2)],
           )

In [82]:
emails.set_index(['week','member'])

EmailsOpened
week       member              
2017-01-08 2                  3
2017-01-15 2                  2
           1                  1

In [83]:
emails.set_index(['week','member']).reindex(complete_idx, fill_value=0)

EmailsOpened
2017-01-08 1             0
           2             3
2017-01-15 1             1
           2             2

In [84]:
all_emails = emails.set_index(['week','member']).reindex(complete_idx, fill_value=0).reset_index()

all_emails.columns = ['week','member','EmailsOpened']
all_emails


,week,member,EmailsOpened
0,2017-01-08,1,0
1,2017-01-08,2,3
2,2017-01-15,1,1
3,2017-01-15,2,2


In [93]:
emails.groupby('member').week.agg(['min','max'])

,min,max
member,,
1,2017-01-15,2017-01-15
2,2017-01-08,2017-01-15


In [85]:
cutoff_dates = emails.groupby('member').week.agg(['min','max']).reset_index()
cutoff_dates

,member,min,max
0,1,2017-01-15,2017-01-15
1,2,2017-01-08,2017-01-15


In [90]:
for _, row in cutoff_dates.iterrows():
  print(row)


member                      1
min       2017-01-15 00:00:00
max       2017-01-15 00:00:00
Name: 0, dtype: object
member                      2
min       2017-01-08 00:00:00
max       2017-01-15 00:00:00
Name: 1, dtype: object


In [91]:
for _, row in cutoff_dates.iterrows():
  member = row['member']
  start_date = row['min']
  end_date = row['max']

  all_emails.drop(all_emails[all_emails.member == member][all_emails.week<start_date].index, inplace=True)
  all_emails.drop(all_emails[all_emails.member == member][all_emails.week>end_date].index, inplace=True)


<ipython-input-91-dbfe8faca9da>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_emails.drop(all_emails[all_emails.member == member][all_emails.week<start_date].index, inplace=True)
<ipython-input-91-dbfe8faca9da>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_emails.drop(all_emails[all_emails.member == member][all_emails.week>end_date].index, inplace=True)


In [92]:
all_emails

,week,member,EmailsOpened
1,2017-01-08,2,3
2,2017-01-15,1,1
3,2017-01-15,2,2


In [105]:
donations = pd.DataFrame(data = [[2, '2017-05-22 11:27:49', 1000],[2, '2017-04-13 09:19:02', 350],[1, '2017-01-01 00:15:45',25]], columns = ['member','timestamp','amount'])

In [106]:
donations

,member,timestamp,amount
0,2,2017-05-22 11:27:49,1000
1,2,2017-04-13 09:19:02,350
2,1,2017-01-01 00:15:45,25


In [107]:
donations.timestamp = pd.to_datetime(donations.timestamp) #datetime 모듈로 으로 변경
donations.set_index('timestamp', inplace = True) #index를 timestamp로 변경
donations

,member,amount
timestamp,,
2017-05-22 11:27:49,2,1000
2017-04-13 09:19:02,2,350
2017-01-01 00:15:45,1,25


In [119]:
agg_don = donations.groupby('member').apply(lambda df: df.amount.resample("W-MON").sum().dropna())
agg_donations = agg_don.reset_index()

In [120]:
agg_donations

,member,timestamp,amount
0,1,2017-01-02,25
1,2,2017-04-17,350
2,2,2017-04-24,0
3,2,2017-05-01,0
4,2,2017-05-08,0
5,2,2017-05-15,0
6,2,2017-05-22,1000


In [111]:
all_emails.groupby('member').count()

,week,EmailsOpened
member,,
1,1,1
2,2,2


In [184]:
merged_df = pd.DataFrame(data=[],columns = ['member','week','EmailsOpened','amount'])
merged_df

,member,week,EmailsOpened,amount


In [185]:
for member, member_email in all_emails.groupby('member'):
  member_donations = agg_donations[agg_donations.member == member] #email 데이터에서 꺼내온 member정보와 매칭되는 agg_donations 데이터프레임을 가져온다.
  member_donations.set_index('timestamp', inplace = True)#index를 column인 timestamp로 변경

  #member_email.set_index('week', inplace = True) #email table의 index를 week로 변경? 필요없는 코드인듯.

  member_email = all_emails[all_emails.member == member] #for 문에서 꺼내온 member에 해당되는 all_emails의 데이터프레임을 가져온다.
  member_email.sort_values('week').set_index('week') #week 정보에 따라 sort한다음 index를 week로 변경

  df = pd.merge(member_email, member_donations, how = 'left', left_index = True, right_index = True)
  df = df.fillna(0) #책 코드 오류인듯. 이렇게 해야 할당된다.

  df['member'] = df.member_x
  merged_df = merged_df.append(df.reset_index()[['member','week','EmailsOpened','amount']])



In [186]:
merged_df

,member,week,EmailsOpened,amount
0,1,2017-01-15,1,0.0
0,2,2017-01-08,3,0.0
1,2,2017-01-15,2,0.0


## 지수평활

In [187]:
import pandas as pd

In [189]:
air = pd.DataFrame([
    ['1949-01', 112],
    ['1949-02', 118],
    ['1949-03', 132],
    ['1949-04', 129],
    ['1949-05', 121],
    ['1949-06', 135],
    ['1949-07', 148],
    ['1949-08', 148],
    ['1949-09', 136],
    ['1949-10', 119],
    ['1949-11', 104],
], columns = ['Date','Passengers'])

In [190]:
air

,Date,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
5,1949-06,135
6,1949-07,148
7,1949-08,148
8,1949-09,136
9,1949-10,119


In [191]:
air['Smooth.5'] = air.ewm(alpha=0.5).mean().Passengers
air['Smooth.1'] = air.ewm(alpha=0.9).mean().Passengers

In [192]:
air

,Date,Passengers,Smooth.5,Smooth.1
0,1949-01,112,112.000000,112.000000
1,1949-02,118,116.000000,117.454545
2,1949-03,132,125.142857,130.558559
3,1949-04,129,127.200000,129.155716
4,1949-05,121,124.000000,121.815498
5,1949-06,135,129.587302,133.681562
6,1949-07,148,138.866142,146.568157
7,1949-08,148,143.450980,147.856816
8,1949-09,136,139.718200,137.185682
9,1949-10,119,129.348974,120.818568


In [14]:
import datetime
import pytz

print(datetime.datetime.utcnow())
print(datetime.datetime.now())
print(datetime.datetime.now(datetime.timezone.utc))

2023-03-09 02:36:49.667878
2023-03-09 02:36:49.669431
2023-03-09 02:36:49.669974+00:00


In [2]:
import datetime
import pytz

datetime.datetime.utcnow()
# datetime.datetime.utcnow()
# datetime.datetime(2018, 5, 31, 14, 49, 43, 187680)

datetime.datetime.now()
# >>> >>> datetime.datetime.now()
# datetime.datetime(2018, 5, 31, 10, 49, 59, 984947)
# as we can see, my computer does not return UTC even though there is no time zone attached

datetime.datetime.now(datetime.timezone.utc)
# >>> datetime.datetime.now(datetime.timezone.utc)
# datetime.datetime(2018, 5, 31, 14, 51, 35, 601355, tzinfo=datetime.timezone.utc)


western = pytz.timezone('US/Pacific')
western.zone
# >>> eastern.zone
# 'US/Pacific'

## the API supports two ways of building a time zone aware time, either via 'localize' or to convert a timezone from one locale to another
# here we localize
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
# datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>>

london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
# >>> london_dt
# datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)
## '2018-05-12 12:15:00 LMT-0001'
## as highlighted in the pytz documentation using the tzinfo of the datetime.datetime initializer does not always lead to the desired outcome
## such as with the London
## according to the pytz documentation, this method does lead to the desired results in time zones without daylight savings

# generally you want to store data in UTC and convert only when generating human readable output
# you can also do date arithmetic with time zones
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1
## this will yield the wrong time delta because the time zones haven't been labelled properly


event1 = london_tz.localize( datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1



event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

## note that in the event you are working on dates for arithmetic that could corss daylight savings time boundaries
## you also need to apply the normalize function for your time zone
event1 = london_tz.localize( datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))

## have a look at pytz.common_timezones
pytz.common_timezones
## or country specific
pytz.country_timezones('RU')
# >>> pytz.country_timezones('RU')
# ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Simferopol', 'Europe/Volgograd', 'Europe/Kirov', 'Europe/Astrakhan', 'Europe/Saratov', 'Europe/Ulyanovsk', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Barnaul', 'Asia/Tomsk', 'Asia/Novokuznetsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Chita', 'Asia/Yakutsk', 'Asia/Khandyga', 'Asia/Vladivostok', 'Asia/Ust-Nera', 'Asia/Magadan', 'Asia/Sakhalin', 'Asia/Srednekolymsk', 'Asia/Kamchatka', 'Asia/Anadyr']
# >>>
# >>> pytz.country_timezones('fr')
# ['Europe/Paris']
# >>>


## time zones
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)
ambig_time.astimezone(western)
ambig_time_later.astimezone(western)
# >>> >>> >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)
# >>> datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)
# >>> >>>
# notice that the last two timestamps are identical, no good!

## in this case you need to use is_dst to indicate whether daylight savings is in effect
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)
ambig_time.astimezone(western)
ambig_time_later.astimezone(western)
# >> >>> datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)


## notice that now we don't have the same time happening twice.
## it may appear that way until you check the offset from UTC
loc_dt.strftime(f)

'2018-05-15 12:34:00 PDT-0700'